In [1]:
from bibleabbr import *

In [2]:
import numpy as np
import shorttext
import sqlite3

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Scripture Loaded

In [3]:
dbconn = sqlite3.connect('ESVCrossway.sqlite')

In [9]:
# cursor = dbconn.cursor()
# result = cursor.execute("select verse, scripture from bible where book is 'ge'")
# for rec in result:
#     print(rec)

In [5]:
from functools import reduce
from nltk import sent_tokenize

def retrieve_docs_as_verses(dbconn):
    cursor = dbconn.cursor()
    for book in wholebible_book_iterator():
        for chap in range(1, numchaps[book]+1):
            chap_label = book+'_'+str(chap)
            result = cursor.execute('select verse, scripture from bible where book is "'+book+'" and chapter='+str(chap))
            for texttuple in result:
                verse = texttuple[0]
                scripture = texttuple[1]
                doc_label = chap_label+':'+str(verse)
                yield doc_label, scripture
    cursor.close()
    

In [6]:
from collections import defaultdict

classdict = defaultdict(lambda : [])
for doc_label, verse in retrieve_docs_as_verses(dbconn):
    classdict[doc_label] = [verse]

In [8]:
len(classdict.keys())

31086

# C-LSTM

In [7]:
wmodel = shorttext.utils.load_word2vec_model('wordembedD/GoogleNews-vectors-negative300.bin')

/usr/local/lib/python3.5/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [10]:
kmodel = shorttext.classifiers.CNNWordEmbed(len(classdict.keys()), vecsize=wmodel.vector_size)

W0630 20:16:24.915434 139944176043776 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0630 20:16:24.941494 139944176043776 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0630 20:16:24.945106 139944176043776 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0630 20:16:24.968838 139944176043776 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0630 20:16:24.994453 139944176043776 deprecation_wrapp

In [11]:
classifier = shorttext.classifiers.VarNNEmbeddedVecClassifier(wmodel)

In [12]:
classifier.train(classdict, kmodel)

W0630 20:23:25.340940 139944176043776 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0630 20:23:25.415295 139944176043776 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/10
31086/31086 [==============================] - 184s 6ms/step - loss: 10.5125
Epoch 2/10
 9728/31086 [========>.....................] - ETA: 2:04 - loss: 10.3025

KeyboardInterrupt: 

In [10]:
classifier.save_compact_model('biblechap_CNN_verse.pkl')

# Prediction

In [4]:
import shorttext

wmodel = shorttext.utils.load_word2vec_model('wordembedD/GoogleNews-vectors-negative300.bin')

/usr/local/lib/python3.5/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [11]:
classifier = shorttext.smartload_compact_model('biblechap_CNN.pkl', wmodel)

In [12]:
sorted(classifier.score('justice').items(), key=lambda item: item[1], reverse=True)[:10]

[('mic_3', 0.5692091),
 ('pr_28', 0.08035246),
 ('ps_37', 0.01926673),
 ('ps_99', 0.016400656),
 ('lk_18', 0.015853051),
 ('ps_82', 0.015455245),
 ('job_29', 0.011227519),
 ('is_56', 0.010322248),
 ('am_5', 0.010122279),
 ('2ch_19', 0.009928787)]